In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import glob
import random
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

import IPython.display as display
from PIL import Image
import cv2

from ultralytics import YOLO

ModuleNotFoundError: No module named 'seaborn'

In [2]:
! pip install ultralytics --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.1 MB 11.7 MB/s eta 0:00:01
   --------------------- ------------------ 4.5/8.1 MB 11.6 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.1 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------------------------------------- -- 2.1/2.3 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ----------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import ultralytics
print(ultralytics.__version__)

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\ADMIN\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
8.3.224


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
### Load pre-trained
model_v11 = YOLO("/content/drive/MyDrive/runs_v2/train/weights/last.pt")

In [ ]:
### Load pre-trained YOLO model
model_v11 = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 96.7MB/s]


In [ ]:
model_v11.train(
    data ="/content/drive/MyDrive/construction safety 1.v3i.yolov11/data.yaml",
    task = 'detect',
    imgsz = 640,
    epochs = 50,
    project = "/content/drive/MyDrive/runs_v3",
    device=0
)

In [ ]:
import cv2
import os
import random
from ultralytics import YOLO

# Load mô hình đã huấn luyện
model = YOLO('./runs_v3/train/weights/best.pt')

# Mở webcam (0 = webcam mặc định, nếu không được thì thử 1 hoặc 2)
cap = cv2.VideoCapture("http://192.168.2.31:4747/video")

# Kiểm tra webcam
if not cap.isOpened():
    print("❌ Không thể mở webcam.")
    exit()

# Tạo thư mục lưu kết quả
os.makedirs('./output_PPE', exist_ok=True)

print("✅ Bắt đầu phát hiện đối tượng. Nhấn 'q' để thoát.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Không đọc được khung hình từ webcam.")
        break

    # Dự đoán với YOLO
    results = model(frame, verbose=False)

    # Lấy frame gốc để vẽ
    annotated_frame = frame.copy()

    if results:
        for box in results[0].boxes:
            class_id = int(box.cls.item())
            confidence = box.conf.item()
            xyxy = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, xyxy)
            label = model.names[class_id]

            # Màu sắc phân loại an toàn / nguy hiểm
            if label in ["no-helmet", "no-vest", "no-boot", "bare-arms"]:
                color = (0, 0, 255)  # Đỏ
            else:
                color = (0, 255, 0)  # Xanh

            # Vẽ khung và nhãn
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, thickness=3)
            text = f"{label} {confidence:.2f}"
            (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_height + 10

            cv2.rectangle(annotated_frame,
                          (text_x - 5, text_y - text_height - 5),
                          (text_x + text_width + 5, text_y + 5),
                          color, -1)
            cv2.putText(annotated_frame, text, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Hiển thị kết quả
    cv2.imshow("YOLO PPE Detection", annotated_frame)

    # Nhấn 's' để lưu khung hình lại
    if cv2.waitKey(1) & 0xFF == ord('s'):
        random_number = random.randint(0, 100000)
        output_path = os.path.join('./output_PPE', f'frame_{random_number}.jpg')
        cv2.imwrite(output_path, annotated_frame)
        print(f"💾 Đã lưu khung hình: {output_path}")

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()
print("✅ Đã thoát.")


❌ Không thể mở webcam.
✅ Bắt đầu phát hiện đối tượng. Nhấn 'q' để thoát.
⚠️ Không đọc được khung hình từ webcam.
✅ Đã thoát.


: 

In [ ]:
import os
import random
import cv2
from ultralytics import YOLO  # Giả định YOLO từ thư viện ultralytics

# Load mô hình đã huấn luyện
model = YOLO('./runs_v3/train/weights/best.pt')  # 'best.pt' là đường dẫn đến mô hình đã được train

# Đọc hình ảnh đầu vào
img = cv2.imread('./image/images.jpg')

# Thực hiện dự đoán trên hình ảnh
results = model(img)

# Lưu kết quả dự đoán vào thư mục output_PPE
os.makedirs('./output_PPE', exist_ok=True)

if results:
    annotated_frame = results[0].plot()
    for box in results[0].boxes:
        class_id = int(box.cls.item())
        confidence = box.conf.item()
        xyxy = box.xyxy[0].tolist()
        x1, y1, x2, y2 = map(int, xyxy)
        label = model.names[class_id]

        # Màu sắc cho nhãn an toàn (màu xanh) và nhãn nguy hiểm (màu đỏ)
        if label == "no-helmet" or label == "no-vest" or label == "no-boot" or label == "bare-arms":
            color = (0, 0, 255)  # Màu đỏ
        else:
            color = (0, 255, 0)  # Màu xanh

        # Vẽ khung với độ dày cố định
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness=3)

        # Cài đặt văn bản và kích thước văn bản cố định
        text = f"{label} {confidence:.2f}"
        text_color = (0, 0, 0)  # Màu đen
        font_scale = 0.6
        font_thickness = 1

        # Tính toán vị trí hiển thị của nhãn
        (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_height + 10

        # Vẽ hình chữ nhật làm nền nhãn để tăng độ tương phản
        cv2.rectangle(img, (text_x - 5, text_y - text_height - 5), (text_x + text_width + 5, text_y + 5), color, -1)
        cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)

# Tạo tên file ngẫu nhiên và lưu hình ảnh kết quả
random_number = random.randint(0, 100000)
output_path = os.path.join('./output_PPE', f'predicted_image_{random_number}.jpg')
cv2.imwrite(output_path, img)
print(f"Hình ảnh đã được lưu tại: {output_path}")


In [ ]:
import os
import random
import cv2
from ultralytics import YOLO

# Load mô hình đã huấn luyện
model = YOLO('/content/drive/MyDrive/runs_v3/train/weights/best.pt')

# Đọc hình ảnh đầu vào
image_paths = [
    '/content/drive/MyDrive/image_test_PPE/young-carpenters-carrying-window-incomplete-house-young-female-male-carpente.jpg',
    '/content/drive/MyDrive/image_test_PPE/pexels-aleksey-2053893-3680959.jpg'
]

# Thư mục để lưu kết quả
output_dir = '/content/drive/MyDrive/output_PPE'
os.makedirs(output_dir, exist_ok=True)

# Hàm điều chỉnh tỷ lệ khung và nhãn
def draw_scaled_box_and_label(img, box, label, confidence, model_names):
    # Tính tỷ lệ dựa trên kích thước ảnh
    img_height, img_width = img.shape[:2]
    thickness = max(2, int(min(img_width, img_height) * 0.005))  # Tỷ lệ dày khung
    font_scale = min(img_width, img_height) / 1000  # Tỷ lệ cỡ chữ

    # Lấy thông tin bounding box
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
    class_id = int(box.cls.item())

    # Thiết lập màu sắc dựa trên loại nhãn
    if model_names[class_id] in ["no-helmet", "no-vest", "no-boot", "bare-arms"]:
        color = (0, 0, 255)  # Màu đỏ
    else:
        color = (0, 255, 0)  # Màu xanh

    # Vẽ bounding box
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness=thickness)

    # Thiết lập văn bản nhãn
    label_text = f"{model_names[class_id]} {confidence:.2f}"
    (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
    text_x, text_y = x1, max(y1 - 10, text_height + 10)

    # Vẽ hình nền cho nhãn để tăng độ tương phản
    cv2.rectangle(img, (text_x - 5, text_y - text_height - 5), (text_x + text_width + 5, text_y + 5), color, -1)
    cv2.putText(img, label_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), thickness=1)

# Thực hiện dự đoán trên từng hình ảnh và lưu kết quả
for image_path in image_paths:
    img = cv2.imread(image_path)
    results = model(img)

    if results:
        for box in results[0].boxes:
            confidence = box.conf.item()
            draw_scaled_box_and_label(img, box, box.cls.item(), confidence, model.names)

    # Tạo tên file ngẫu nhiên và lưu hình ảnh kết quả
    random_number = random.randint(0, 100000)
    output_path = os.path.join(output_dir, f'predicted_image_{random_number}.jpg')
    cv2.imwrite(output_path, img)
    print(f"Hình ảnh đã được lưu tại: {output_path}")



0: 448x640 4 Glovess, 2 Helmets, 2 Persons, 2 Safety Vests, 12.5ms
Speed: 2.7ms preprocess, 12.5ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Hình ảnh đã được lưu tại: /content/drive/MyDrive/output_PPE/predicted_image_67109.jpg

0: 448x640 3 Glovess, 2 Helmets, 2 Persons, 2 bare-armss, 3 no-vests, 14.7ms
Speed: 4.0ms preprocess, 14.7ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)
Hình ảnh đã được lưu tại: /content/drive/MyDrive/output_PPE/predicted_image_80854.jpg


In [ ]:
# prompt: tạo cho tôi đoạn mã nhận diện như trên với đầu vào là video nhưng nếu nhãn là non-helmet thì sử dụng viền màu đỏ còn các nhãn khác dùng màu xanh

# Load mô hình đã huấn luyện
model = YOLO('/content/drive/MyDrive/runs_v3/train/weights/best.pt')  # 'best.pt' là đường dẫn đến mô hình đã được train

# Đường dẫn đến video đầu vào
video_path = '/content/drive/MyDrive/image_test_PPE/5950179514373.mp4'  # Thay thế bằng đường dẫn thực tế của video
random_number = random.randint(0, 100000)
# Đường dẫn để lưu video đầu ra
output_video_path = f'/content/drive/MyDrive/output_PPE/output_video_{random_number}.mp4'  # Thay thế bằng đường dẫn mong muốn

# Mở video đầu vào
cap = cv2.VideoCapture(video_path)

# Kiểm tra xem video có mở được không
if not cap.isOpened():
    print("Không thể mở video")
    exit()
frame_count = 0
# Lấy thông tin về kích thước khung hình và FPS của video đầu vào
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

# Tạo đối tượng VideoWriter để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Sử dụng codec mp4v
out = cv2.VideoWriter(output_video_path, fourcc, fps, (8192, 5464))

while True:
    # Đọc từng khung hình từ video
    ret, frame = cap.read()
    # Kiểm tra xem còn khung hình nào không
    if not ret:
        break
    frame = cv2.resize(frame, (8192, 5464), interpolation=cv2.INTER_LINEAR)
    results = model(frame)

    # Hiển thị kết quả dự đoán trên khung hình
    if results:
        annotated_frame = results[0].plot()
        for box in results[0].boxes:
            class_id = int(box.cls.item())  # Chuyển đổi class_id thành số nguyên
            confidence = box.conf.item()
            xyxy = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, xyxy)
            label = model.names[class_id]

            if label == "no-helmet" or label == "no-vest" or label == "no-boot" or label == "bare-arms":  # Thay "no_helmet" bằng tên class thực tế
                color = (0, 0, 255)  # Red
            else:
                color = (0, 255, 0)  # Green

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 40)

            # Text settings
            text_color = (0, 0, 0)  # Black text

            # Calculate text size and position for centering
            (text_width, text_height), _ = cv2.getTextSize(f"{label} {confidence:.2f}", cv2.FONT_HERSHEY_SIMPLEX, 8, 100)
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_height + 10

            # Draw filled rectangle behind the text for better visibility
            cv2.rectangle(frame, (text_x -20, text_y - text_height - 20), (text_x + text_width, text_y + 5), color, -1)
            cv2.putText(frame, f"{label} {confidence:.2f}", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 8, text_color, 20)


    # Ghi khung hình đã được chú thích vào video đầu ra
    out.write(frame)
    frame_count += 1
print(f"Frame {frame_count} added to output video")
# Giải phóng tài nguyên
cap.release()
out.release()

print(f"Video đã được lưu vào: {output_video_path}")


0: 448x640 2 Helmets, 2 Persons, 2 Safety Vests, 3 bare-armss, 194.2ms
Speed: 8.5ms preprocess, 194.2ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 Helmets, 2 Persons, 2 Safety Vests, 3 bare-armss, 169.5ms
Speed: 8.3ms preprocess, 169.5ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 Helmets, 2 Persons, 2 Safety Vests, 3 bare-armss, 182.0ms
Speed: 6.0ms preprocess, 182.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 Helmets, 2 Persons, 2 Safety Vests, 3 bare-armss, 187.3ms
Speed: 8.2ms preprocess, 187.3ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 Helmets, 2 Persons, 2 Safety Vests, 3 bare-armss, 216.5ms
Speed: 7.6ms preprocess, 216.5ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 Helmets, 2 Persons, 2 Safety Vests, 4 bare-armss, 1 no-vest, 164.5ms
Speed: 7.4ms preprocess, 164.5ms inference, 1.0ms postprocess per i

In [8]:
# prompt: tạo cho tôi đoạn mã nhận diện như trên với đầu vào là video nhưng nếu nhãn là non-helmet thì sử dụng viền màu đỏ còn các nhãn khác dùng màu xanh

# Load mô hình đã huấn luyện
model = YOLO('/content/drive/MyDrive/runs_v3/train/weights/best.pt')  # 'best.pt' là đường dẫn đến mô hình đã được train

# Đường dẫn đến video đầu vào
video_path = '/content/drive/MyDrive/image_test_PPE/5950180490638.mp4'  # Thay thế bằng đường dẫn thực tế của video
random_number = random.randint(0, 100000)
# Đường dẫn để lưu video đầu ra
output_video_path = f'/content/drive/MyDrive/output_PPE/output_video_{random_number}.mp4'  # Thay thế bằng đường dẫn mong muốn

# Mở video đầu vào
cap = cv2.VideoCapture(video_path)

# Kiểm tra xem video có mở được không
if not cap.isOpened():
    print("Không thể mở video")
    exit()
frame_count = 0
# Lấy thông tin về kích thước khung hình và FPS của video đầu vào
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

# Tạo đối tượng VideoWriter để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Sử dụng codec mp4v
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    # Đọc từng khung hình từ video
    ret, frame = cap.read()
    # Kiểm tra xem còn khung hình nào không
    if not ret:
        break
    results = model(frame)

    # Hiển thị kết quả dự đoán trên khung hình
    if results:
        annotated_frame = results[0].plot()
        for box in results[0].boxes:
            class_id = int(box.cls.item())  # Chuyển đổi class_id thành số nguyên
            confidence = box.conf.item()
            xyxy = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, xyxy)
            label = model.names[class_id]

            if label == "no-helmet" or label == "no-vest" or label == "no-boot" or label == "bare-arms":  # Thay "no_helmet" bằng tên class thực tế
                color = (0, 0, 255)  # Red
            else:
                color = (0, 255, 0)  # Green

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

            # Text settings
            text_color = (0, 0, 0)  # Black text

            # Calculate text size and position for centering
            (text_width, text_height), _ = cv2.getTextSize(f"{label} {confidence:.2f}", cv2.FONT_HERSHEY_SIMPLEX, 1, 6)
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_height + 10

            # Draw filled rectangle behind the text for better visibility
            cv2.rectangle(frame, (text_x, text_y - text_height - 5), (text_x + text_width, text_y + 5), color, -1)
            cv2.putText(frame, f"{label} {confidence:.2f}", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 3)


    # Ghi khung hình đã được chú thích vào video đầu ra
    out.write(frame)
    frame_count += 1
print(f"Frame {frame_count} added to output video")
# Giải phóng tài nguyên
cap.release()
out.release()

print(f"Video đã được lưu vào: {output_video_path}")


0: 384x640 2 Glovess, 1 Person, 1 Safety Boot, 1 no-helmet, 137.5ms
Speed: 4.8ms preprocess, 137.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Glovess, 1 Person, 1 Safety Boot, 1 no-helmet, 126.6ms
Speed: 4.3ms preprocess, 126.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gloves, 1 Person, 1 Safety Boot, 1 no-helmet, 137.1ms
Speed: 3.9ms preprocess, 137.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gloves, 1 Person, 1 Safety Boot, 1 no-helmet, 136.4ms
Speed: 3.5ms preprocess, 136.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gloves, 1 Person, 1 Safety Boot, 1 no-helmet, 203.1ms
Speed: 5.4ms preprocess, 203.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Gloves, 1 Person, 1 no-helmet, 196.1ms
Speed: 3.5ms preprocess, 196.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Glo

In [ ]:
# prompt: tạo cho tôi đoạn mã có chức năng tương tự như ô trên nhưng đầu vào là từ camera máy tính và đầu ra hiển thị theo thời gian thực

import cv2
from ultralytics import YOLO

# Load mô hình đã huấn luyện
model = YOLO('/content/drive/MyDrive/runs_v3/train/weights/best.pt')

# Khởi tạo camera
cap = cv2.VideoCapture(0)  # Sử dụng camera mặc định (0)

if not cap.isOpened():
    print("Không thể mở camera")
    exit()

while True:
    # Đọc khung hình từ camera
    ret, frame = cap.read()
    if not ret:
        break

    # Thực hiện dự đoán trên khung hình
    results = model(frame)

    # Hiển thị kết quả dự đoán trên khung hình
    if results:
        annotated_frame = results[0].plot()
        cv2.imshow('YOLOv8 Inference', annotated_frame)

    # Thoát khỏi vòng lặp khi nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()